In [1]:
!pip install -q sagemaker sagemaker[local]

# Install `transformers` from master
!pip install -q git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers|sagemaker'

You are using pip version 19.0.3, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.3, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
sagemaker           1.60.2    
tokenizers          0.7.0     
transformers        2.11.0    
You are using pip version 19.0.3, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/hunkim-transformer'

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='hunkimSagemaker')['Role']['Arn']
    
print(role)

arn:aws:iam::294038372338:role/hunkimSagemaker


In [3]:
# in this notebook we'll only get one of the files (the Oscar one) for the sake of simplicity and performance
!mkdir EsperBERTo

!wget -O EsperBERTo/oscar.eo.txt -c https://cdn-datasets.huggingface.co/EsperBERTo/data/oscar.eo.txt 


mkdir: EsperBERTo: File exists
--2020-06-10 23:44:08--  https://cdn-datasets.huggingface.co/EsperBERTo/data/oscar.eo.txt
Resolving cdn-datasets.huggingface.co (cdn-datasets.huggingface.co)...52.85.193.97, 52.85.193.51, 52.85.193.66, ...
Connecting to cdn-datasets.huggingface.co (cdn-datasets.huggingface.co)|52.85.193.97|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [4]:
inputs = sagemaker_session.upload_data(path='EsperBERTo', bucket=bucket, key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(inputs))

input spec (in this case, just an S3 path): s3://sagemaker-us-west-2-294038372338/sagemaker/hunkim-transformer


In [5]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='train.py',
                    role=role,
                    framework_version='1.5.0',
                    train_instance_count=1,
                    train_instance_type='ml.p3.2xlarge',
                    #train_instance_type='local',
                    source_dir='code',
                    hyperparameters={
                        'epochs': 1,
                        'tied': True
                    })

In [6]:
%%time
estimator.fit({'training': inputs})

ully installed default-user-module-name-1.0.0 filelock-3.0.12 regex-2020.6.8 sacremoses-0.0.43 sentencepiece-0.1.91 tokenizers-0.7.0 transformers-2.11.0
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
2020-06-10 14:57:04,927 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "training": "/opt/ml/input/data/training"
    },
    "current_host": "algo-1",
    "framework_module": "sagemaker_pytorch_container.training:main",
    "hosts": [
        "algo-1"
    ],
    "hyperparameters": {
        "tied": true,
        "epochs": 1
    },
    "input_config_dir": "/opt/ml/input/config",
    "input_data_config": {
        "training": {
            "TrainingInputMode": "File",
            "S3DistributionType": "FullyReplicated",
            "RecordWrapperType": "None"
        }
    },
    "input_dir": "/opt/ml/input",
    "is_master": true,
    "job_

In [7]:
training_job_name = estimator.latest_training_job.name
desc = sagemaker_session.sagemaker_client.describe_training_job(TrainingJobName=training_job_name)
trained_model_location = desc['ModelArtifacts']['S3ModelArtifacts']
print(trained_model_location)
# s3://sagemaker-us-west-2-294038372338/pytorch-training-2020-06-10-11-15-27-506/output/model.tar.gz

s3://sagemaker-us-west-2-294038372338/pytorch-training-2020-06-10-14-44-46-074/output/model.tar.gz


In [8]:
from sagemaker.predictor import RealTimePredictor, json_serializer, json_deserializer

class JSONPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(JSONPredictor, self).__init__(endpoint_name, sagemaker_session, json_serializer, json_deserializer)

In [9]:
from sagemaker.pytorch import PyTorchModel

training_job_name = estimator.latest_training_job.name
desc = sagemaker_session.sagemaker_client.describe_training_job(TrainingJobName=training_job_name)
trained_model_location = desc['ModelArtifacts']['S3ModelArtifacts']

model = PyTorchModel(model_data=trained_model_location,
                     role=role,
                     framework_version='1.5.0',
                     entry_point='infer.py',
                     source_dir='code',
                     predictor_cls=JSONPredictor)

In [10]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

# Get the end point
endpoint = predictor.endpoint
print(endpoint)

----------------!pytorch-inference-2020-06-10-15-05-28-352


In [11]:
#FIXME: This works, but the predictor using endpoint does not work
input = {
    'text': "La suno <mask>."
}
response = predictor.predict(input)
print(response)

[{'sequence': '<s> La sunomal.</s>', 'score': 0.016554249450564384, 'token': 1555}, {'sequence': '<s> La sunoa.</s>', 'score': 0.011879165656864643, 'token': 69}, {'sequence': '<s> La sunoÅĿ.</s>', 'score': 0.009716324508190155, 'token': 16886}, {'sequence': '<s> La sunore.</s>', 'score': 0.008137395605444908, 'token': 286}, {'sequence': '<s> La sunos.</s>', 'score': 0.007335992064327002, 'token': 87}]


In [12]:
import boto3
import json

client = boto3.client('sagemaker-runtime')

input = {
    'text': "La suno <mask>."
}

content_type = "application/json "                                     
accept = "application/json"                                       
payload = json.dumps(input)

response = client.invoke_endpoint(
    EndpointName=endpoint, 
    ContentType=content_type,
    Accept=accept,
    Body=payload
    )

print(response['Body'].read())  

b'[{"sequence": "<s> La sunomal.</s>", "score": 0.016554249450564384, "token": 1555}, {"sequence": "<s> La sunoa.</s>", "score": 0.011879165656864643, "token": 69}, {"sequence": "<s> La suno\\u00c5\\u013f.</s>", "score": 0.009716324508190155, "token": 16886}, {"sequence": "<s> La sunore.</s>", "score": 0.008137395605444908, "token": 286}, {"sequence": "<s> La sunos.</s>", "score": 0.007335992064327002, "token": 87}]'


In [13]:
sagemaker_session.delete_endpoint(predictor.endpoint)